# Database loader
This notebook provides a step-by-step guide for loading data into an ArangoDB database.

In [44]:
# 1. Install dependencies
#! pip install python-arango --upgrade

In [45]:
# 2. Import modules
import os
import json
from arango import ArangoClient

import pandas as pd 
import plotly.express as px
from datetime import datetime
import hashlib

In [46]:
# 3. Define file paths
db_dir = '../output'
db_nodes_path = os.path.join(db_dir, "fermentdb_nodes_full.json")
db_edges_path = os.path.join(db_dir, "fermentdb_edges_full.json")

### 1. Load data: nodes and edges dictionaries. 

In [47]:
# Load nodes json
with open(db_nodes_path, 'r') as dbfile:
    collections_str = dbfile.read()

nodes = json.loads(collections_str)

In [48]:
# Load edges json
with open(db_edges_path, 'r') as dbfile:
    edges_str = dbfile.read()

edges = json.loads(edges_str)

### 2. Connect to ArangoDB

In [49]:
# Initialize the client for ArangoDB.
# client = ArangoClient(hosts="http://localhost:8529")

# Connect to "_system" database as root user.
sys_db = client.db("_system", username="root", password= "")
# sys_db = client.db("_system", username="root", password="psswrd")

# Create a new database named "fermentdb" if it does not exist.
if not sys_db.has_database('fermentdb'):
    sys_db.create_database('fermentdb')

# Retrieve the names of all databases on the server as list of strings
db_list = sys_db.databases()
print(db_list)

['_system', 'fermentdb']


### 3. Connect to FermentDB and create collections: nodes and edges

In [50]:
# Connect to "fermentdb" database as root user.
# db = client.db("fermentdb", username="root", password="psswrd")

In [51]:
# Create graph fermentdb if it does not exist.
if not db.has_graph('fermentdb'):
    db.create_graph('fermentdb')

fermentdb = db.graph('fermentdb')

In [52]:
# db.delete_graph('fermentdb', drop_collections=True)

In [53]:
# Check graph
db.graphs()
print("----")
fermentdb.name
fermentdb.db_name
fermentdb.vertex_collections()
print("----")
fermentdb.edge_definitions()

----
----


[{'edge_collection': 'belongs_to',
  'from_vertex_collections': ['Strain'],
  'to_vertex_collections': ['Species']},
 {'edge_collection': 'created_at',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['Institution']},
 {'edge_collection': 'created_by',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['User']},
 {'edge_collection': 'cultures_strain',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Strain']},
 {'edge_collection': 'from',
  'from_vertex_collections': ['Institution'],
  'to_vertex_collections': ['Country']},
 {'edge_collection': 'has_batch',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['Batch_cell_culture']},
 {'edge_collection': 'has_condition',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Process_condition']},
 {'edge_collection': 'has_initial_condition',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Initial_condition']},
 {'edge_collection': '

In [54]:
# Load node/vertex collections
for node in nodes:
    if fermentdb.has_vertex_collection(node):
        v = fermentdb.vertex_collection(node)
    else:
        v = fermentdb.create_vertex_collection(node)

    if type(nodes[node]) != list:
        nodes[node] = [nodes[node]]
    
    v.insert_many(nodes[node])

# List vertex collections in the graph.
fermentdb.vertex_collections()

['Batch_cell_culture',
 'Country',
 'Fermenter',
 'Initial_condition',
 'Institution',
 'Phase_event',
 'Process_condition',
 'Project',
 'Run',
 'Species',
 'Strain',
 'User']

In [55]:
# Load edges collections
for edge in edges:
    if fermentdb.has_edge_definition(edge):
        e = fermentdb.edge_collection(edge)
    else:
        e = fermentdb.create_edge_definition(
            edge_collection=edge,
            from_vertex_collections=edges[edge]['from_collection'],
            to_vertex_collections=edges[edge]['to_collection']
        )
    # if not fermentdb.has_edge_definition(edge):
    #     e = fermentdb.create_edge_definition(
    #         edge_collection=edge,
    #         from_vertex_collections=edges[edge]['from_collection'],
    #         to_vertex_collections=edges[edge]['to_collection']
    #     )
    for l in edges[edge]['edges']:
        e.insert(l)
    
    # List edge definitions.
fermentdb.edge_definitions()

[{'edge_collection': 'belongs_to',
  'from_vertex_collections': ['Strain'],
  'to_vertex_collections': ['Species']},
 {'edge_collection': 'created_at',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['Institution']},
 {'edge_collection': 'created_by',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['User']},
 {'edge_collection': 'cultures_strain',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Strain']},
 {'edge_collection': 'from',
  'from_vertex_collections': ['Institution'],
  'to_vertex_collections': ['Country']},
 {'edge_collection': 'has_batch',
  'from_vertex_collections': ['Project'],
  'to_vertex_collections': ['Batch_cell_culture']},
 {'edge_collection': 'has_condition',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Process_condition']},
 {'edge_collection': 'has_initial_condition',
  'from_vertex_collections': ['Run'],
  'to_vertex_collections': ['Initial_condition']},
 {'edge_collection': '

### 4. AQL Queries

In [56]:
# Function Definitions

def get_hash(key, prefix=""):
    hkey = str(int(hashlib.sha1(key.encode("utf-8")).hexdigest(), 16) % (10 ** 8))
    hkey = f"{prefix}{hkey}"
    
    return hkey

def get_condition_data(strain, condition,fermenter):
    condition = [f"Process_condition/{get_hash(c, prefix='C')}" for c in condition]
    
    query = '''FOR doc IN Run
      FILTER doc.strain_batch == @val AND doc.container_type == @fermenter
      FOR v, e IN 1..1 OUTBOUND doc has_condition
        FILTER e._to IN @condition
        RETURN { source: doc, target: v, edge: e }
    '''
                   
    cursor = db.aql.execute(query,bind_vars={'val': strain,
                              'condition':condition, 'fermenter': fermenter})

    
    result = [doc for doc in cursor]

    return result


In [57]:
# def get_hash(key, prefix=""):
#     hkey = str(int(hashlib.sha1(key.encode("utf-8")).hexdigest(), 16) % (10 ** 8))
#     hkey = f"{prefix}{hkey}"
    
#     return hkey

# def get_condition_data(strain, condition, fermenter):
#     condition = [f"Process_condition/{get_hash(c, prefix='C')}" for c in condition]
#     strain = [f"Strain/{get_hash(s, prefix='S')}" for s in strain]
#     fermenter = f"Fermenter/{fermenter}"

#     query = '''FOR doc IN Run
#       FOR v, e IN 1..1 OUTBOUND doc uses_fermenter
#         FILTER e._to == @fermenter
#         FOR ve, ed IN 1..1 OUTBOUND doc cultures_strain
#           FILTER ed._to IN @strain
#           FOR vertex, edge IN 1..1 OUTBOUND doc has_condition
#             FILTER edge._to IN @condition
#             RETURN { source: doc, target: vertex, edge: edge }
#     '''
#                  
#     cursor = db.aql.execute(query,bind_vars={'strain': strain,
#                               'condition':condition, 'fermenter': fermenter})

    
#     result = [doc for doc in cursor]

#     return result

### 5. Plot conditions

In [58]:
# Function Definitions

def plot_condition(strain, condition, fermenter):
    result = get_condition_data(strain, condition, fermenter)
    rows = []
    for r in result:
        source = r['source']['_key']
        target = r['target']['name']
        data = r['edge']['data']
        timestamps = r['edge']['timestamps']
        rows.append(pd.DataFrame({'run': source, 'data':data, 'time': timestamps, 'condition': target}))
    
    df = pd.concat(rows)
    df['time'] = df['time'].apply(lambda t: datetime.fromtimestamp(t))
    df = df.sort_values(by="time")
    fig = px.line(df, x="time", y="data", color='run', line_dash='condition')
    fig.show(renderer='notebook')
    # fig.show(renderer='iframe')

In [ ]:
# Plot condition

plot_condition(strain="Strain1", condition=["D-glucose"], fermenter="AMBR 250")
# plot_condition(strain="Strain1", condition=["D-glucose"], fermenter="Ambr250")